In [309]:
from selenium import webdriver
import pandas as pd
import time

In [311]:
url="https://race.netkeiba.com/odds/index.html?race_id=202005040801&rf=race_submenu"
driver = webdriver.Chrome('C:\\Users\\shun_k\\chromedriver\\chromedriver')#chromedriverの起動
driver.get(url)#読み込み
time.sleep(10)#ページ読み込み待機

# 単勝，複勝オッズのスクレイピング

In [313]:
html = driver.page_source
driver.close()
odds_df=pd.read_html(html)[0]

InvalidSessionIdException: Message: invalid session id


# 出馬表のスクレイピング

In [338]:
url = "https://race.netkeiba.com/race/shutuba.html?race_id=202005040801&rf=race_submenu"
driver = webdriver.Chrome('C:\\Users\\shun_k\\chromedriver\\chromedriver')
driver.get(url)
time.sleep(10)
html = driver.page_source
shutuba_df = pd.read_html(html,header=1)[0]
driver.close()

In [356]:
test = shutuba_df
test

,枠,馬番,印,馬名,性齢,斤量,騎手,厩舎,馬体重(増減),オッズ,人気,登録,メモ
0,1,1,--,シゲルセンチュリー,牝2,54.0,柴田善,美浦石毛,462(+2),27.6,6,NaN,NaN
1,2,2,--,ナンヨーノワール,牡2,55.0,木幡巧,美浦杉浦,448(-2),80.4,10,NaN,NaN
2,3,3,--,サノノバーディ,牡2,55.0,田辺,美浦尾形,456(-2),72.8,9,NaN,NaN
3,4,4,--,カイザーレヴァンテ,牡2,55.0,Ｍデムーロ,美浦田中剛,488(-4),43.8,7,NaN,NaN
4,4,5,--,アルビレオ,牡2,55.0,武藤,美浦林,482(-2),198.1,11,NaN,NaN
5,5,6,--,トーセンダニエル,牡2,55.0,武士沢,美浦古賀史,548(+8),3.4,2,NaN,NaN
6,5,7,--,センナリ,牡2,54.0,木幡育,美浦大江原,482(-4),489.0,13,NaN,NaN
7,6,8,--,インヴァリアンス,牡2,55.0,横山典,美浦尾関,482(+4),7.3,4,NaN,NaN
8,6,9,--,マイネルホップ,牡2,55.0,柴田大,美浦伊藤圭,444(+4),299.5,12,NaN,NaN
9,7,10,--,ロクネンニクミ,牡2,55.0,内田博,美浦小笠,516(0),18.7,5,NaN,NaN


test.columns

# test.rename(columns={'馬体重(増減)': "馬体重"},inplace=True)
test

In [357]:
def processing_shutubatable(results):
    # results = pd.read_pickle("results.pickle")
    df = results.copy()
    # 着順
    # df = df[~(df["着順"].astype(str).str.contains("\D"))]
    # df["着順"] = df["着順"].astype(int)
    # 性別
    df["性別"] = df["性齢"].map(lambda x: str(x)[0])
    df["性別"] = df["性別"].map({"牡": 1, "牝": 0, "セ": 1})  # 雄:1,牝:0
    # df["性別"]=df["性別"].astype(int)
    # df["性別"] = pd.get_dummies(df["性別"])
    # 年齢
    df["年齢"] = df["性齢"].map(lambda x: str(x)[1:])
    df["年齢"] = df["年齢"].astype(int)
    # 体重
    df = df.rename(columns={'馬体重(増減)': "馬体重"})
    df["体重"] = df["馬体重"].str.split("(", expand=True)[0].astype(int)
    # 体重変化
    df["体重変化"] = df["馬体重"].str.split("(", expand=True)[1].str[:-1].astype(int)
    # 人気と単勝をint型に
    df["人気"] = df["人気"].astype(int)
    # df["単勝"] = df["単勝"].astype(float)
    df = df.rename(columns={'オッズ': "単勝"})
    df = df.rename(columns={'枠': "枠番"})
    # いらない列の消去
    df.drop(["印","性齢","馬名", "厩舎", "馬体重", "登録", "メモ"], axis=1, inplace=True)
    #人気順にソート
    df = df.sort_values(by="人気",ascending=True)

    return df

In [358]:
test = processing_shutubatable(test)

,枠番,馬番,斤量,騎手,単勝,人気,性別,年齢,体重,体重変化
12,8,13,55.0,北村宏,3.2,1,1,2,546,4
5,5,6,55.0,武士沢,3.4,2,1,2,548,8
11,8,12,55.0,ルメール,3.5,3,1,2,468,8
7,6,8,55.0,横山典,7.3,4,1,2,482,4
9,7,10,55.0,内田博,18.7,5,1,2,516,0
0,1,1,54.0,柴田善,27.6,6,0,2,462,2
3,4,4,55.0,Ｍデムーロ,43.8,7,1,2,488,-4
10,7,11,55.0,大野,57.9,8,1,2,488,6
2,3,3,55.0,田辺,72.8,9,1,2,456,-2
1,2,2,55.0,木幡巧,80.4,10,1,2,448,-2


In [362]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13 entries, 12 to 6
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   枠番      13 non-null     int64  
 1   馬番      13 non-null     int64  
 2   斤量      13 non-null     float64
 3   騎手      13 non-null     object 
 4   単勝      13 non-null     float64
 5   人気      13 non-null     int32  
 6   性別      13 non-null     int64  
 7   年齢      13 non-null     int32  
 8   体重      13 non-null     int32  
 9   体重変化    13 non-null     int32  
dtypes: float64(2), int32(4), int64(3), object(1)
memory usage: 1.5+ KB


In [363]:
import pickle
filename="finalized_model.sav"
model = pickle.load(open(filename, 'rb'))

In [371]:
from sklearn.preprocessing import OneHotEncoder
X_train = pd.read_pickle(X_train.pickle)
ohe = OneHotEncoder(handle_unknown="ignore", sparse=False)
# ohe.fit(X_train)

NameError: name 'X_train' is not defined

In [399]:
df = pd.read_pickle("organized_results.pickle")
df
df_concat = pd.concat([test, df])
df_concat

,枠番,馬番,斤量,騎手,単勝,人気,性別,年齢,体重,体重変化,着順,馬名
12,8,13,55.0,北村宏,3.2,1,1,2,546,4,NaN,NaN
5,5,6,55.0,武士沢,3.4,2,1,2,548,8,NaN,NaN
11,8,12,55.0,ルメール,3.5,3,1,2,468,8,NaN,NaN
7,6,8,55.0,横山典,7.3,4,1,2,482,4,NaN,NaN
9,7,10,55.0,内田博,18.7,5,1,2,516,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
201910020812,7,12,52.0,大野拓弥,45.7,10,0,3,414,6,10.0,オーシャンスケイプ
201910020812,3,3,55.0,藤岡康太,29.8,9,0,4,486,2,11.0,プリヴェット
201910020812,5,7,55.0,松山弘平,29.6,8,0,4,476,6,12.0,アドマイヤクィーン
201910020812,8,13,49.0,亀田温心,134.8,12,0,3,472,2,13.0,タムロドリーム


In [395]:
df_concat.columns

Index(['枠番', '馬番', '斤量', '騎手', '単勝', '人気', '性別', '年齢', '体重', '体重変化', '着順',
       '馬名'],
      dtype='object')

In [404]:
df_concat.drop(columns=["着順","馬名"],inplace=True)

In [412]:
df_concat["騎手"].describe()

count     43324
unique      199
top        和田竜二
freq        822
Name: 騎手, dtype: object

In [424]:
test["騎手"].head(14)

12      北村宏
5       武士沢
11     ルメール
7       横山典
9       内田博
0       柴田善
3     Ｍデムーロ
10       大野
2        田辺
1       木幡巧
4        武藤
8       柴田大
6       木幡育
Name: 騎手, dtype: object

In [422]:
df[df["騎手"]=="木幡育"]

,着順,枠番,馬番,馬名,斤量,騎手,単勝,人気,性別,年齢,体重,体重変化


In [405]:
X=pd.get_dummies(df_concat).head(13)


In [406]:
X

,枠番,馬番,斤量,単勝,人気,性別,年齢,体重,体重変化,騎手_アヴドゥ,...,騎手_高倉稜,騎手_高松亮,騎手_高田潤,騎手_高野和馬,騎手_鮫島克駿,騎手_鮫島良太,騎手_黒岩悠,騎手_黛弘人,騎手_Ｍデムーロ,騎手_Ｍ．デム
12,8,13,55.0,3.2,1,1,2,546,4,0,...,0,0,0,0,0,0,0,0,0,0
5,5,6,55.0,3.4,2,1,2,548,8,0,...,0,0,0,0,0,0,0,0,0,0
11,8,12,55.0,3.5,3,1,2,468,8,0,...,0,0,0,0,0,0,0,0,0,0
7,6,8,55.0,7.3,4,1,2,482,4,0,...,0,0,0,0,0,0,0,0,0,0
9,7,10,55.0,18.7,5,1,2,516,0,0,...,0,0,0,0,0,0,0,0,0,0
0,1,1,54.0,27.6,6,0,2,462,2,0,...,0,0,0,0,0,0,0,0,0,0
3,4,4,55.0,43.8,7,1,2,488,-4,0,...,0,0,0,0,0,0,0,0,1,0
10,7,11,55.0,57.9,8,1,2,488,6,0,...,0,0,0,0,0,0,0,0,0,0
2,3,3,55.0,72.8,9,1,2,456,-2,0,...,0,0,0,0,0,0,0,0,0,0
1,2,2,55.0,80.4,10,1,2,448,-2,0,...,0,0,0,0,0,0,0,0,0,0


In [408]:
# X = pd.get_dummies(test)
model.predict(X)
# X

ValueError: X has 208 features per sample; expecting 196

In [369]:
model.predict(X)

ValueError: X has 22 features per sample; expecting 196